In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time

In [2]:
# base_price=[[10,10,10],
#             [10,10,10]]

# intercept = [23,20]
# coef_1 =[-6,-5]
# coef_2 =[0.01,0.02]
# coef_3 = [1,1.2]
# coef_4 = [-0.003,-0.004]
products=2
coef_p1 = [-6,0.01,1,-0.003,23]
coef_p2 = [1.2,-0.004,-5,0.02,20]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]
base_price = [base_price_1,base_price_2]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
flag_p1=[1,0,0]
flag_p2=[0,0,1]

target_trade_spend = [276860,223573]
total_target_trade_spend_final = sum(target_trade_spend)

target_edlp_spend = [63366,35525]
target_tpr_spend = [213494,188048]

idx = range(6)
products = 2
weeks = 3
tot = products*weeks

In [3]:
target_trade_spend_lower_bd = [x * 0.9 for x in target_trade_spend]
target_trade_spend_upper_bd = [x * 1.1 for x in target_trade_spend]

target_edlp_spend_lower_bd = [x * 0.8 for x in target_edlp_spend]
target_edlp_spend_upper_bd = [x * 1.2 for x in target_edlp_spend]

target_tpr_spend_lower_bd = [x * 0.8 for x in target_tpr_spend]
target_tpr_spend_upper_bd = [x * 1.2 for x in target_tpr_spend]

In [4]:
def cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    
    unit_sales = []
    for i in  range(weeks):
#         print(edlp_p1[i],coef[0],flag_p1[i])
        a = (log( edlp_p1[i])*coef[0])* flag_p1[i]
        b = (log(base_price_1[i])*coef[0]+ tpr_p1[i]*coef[1])*(1- flag_p1[i])
        
        c = (log( edlp_p2[i])*coef[2])* flag_p2[i]
        d = (log(base_price_2[i])*coef[2]+ tpr_p2[i]*coef[3])*(1- flag_p2[i])
        
#         print(a,' $ ',b,' $ ',c,' $ ',d,' $ ',coef[4])
        unit_sales.append(exp(a+b+c+d+coef[4]))                 
   
    return unit_sales

def calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j):
    price = []
    edlp = [edlp_p1,edlp_p2]
    tpr = [tpr_p1,tpr_p2]
    flag = [flag_p1,flag_p2]
    base_price=[base_price_1,base_price_2]
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i]/100)*(1-flag[j][i]))  +  (edlp[j][i]*flag[j][i]))
                   
    return price
                   
                   
def calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    dollar_sales=[]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    edlp_trade_spent = []
    flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


In [5]:
def create_model(init):
    print("Init value= ",init)
    def initial(model,i):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    
    model.edlp_p1 = Var(range(weeks), initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr_p1 = Var(range(weeks), initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag_p1 = Var(range(weeks),initialize=initial,domain=Binary)
    
    model.edlp_p2 = Var(range(weeks), initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr_p2 = Var(range(weeks), initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag_p2 = Var(range(weeks),initialize=initial,domain=Binary)
    
    dollar_sales = calc_dollar_sales(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    total_trade_spent = calc_total_trade_spent(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    edlp_trade_spent = calc_edlp_trade_spent(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    
    model.obj = Objective(expr=sum(dollar_sales[0])+sum(dollar_sales[1]) ,sense= maximize)
    model.c1 = Constraint(expr = sum(total_trade_spent[0]) + sum(total_trade_spent[1]) == total_target_trade_spend_final)
    
    model.c2 = Constraint(expr = sum(total_trade_spent[0]) >= target_trade_spend_lower_bd[0])
    model.c3 = Constraint(expr = sum(total_trade_spent[0]) <= target_trade_spend_upper_bd[0])
    
    model.c4 = Constraint(expr = sum(total_trade_spent[1]) >= target_trade_spend_lower_bd[1])
    model.c5 = Constraint(expr = sum(total_trade_spent[1]) <= target_trade_spend_upper_bd[1])
    
    
    model.c6 = Constraint(expr = sum(edlp_trade_spent[0]) >= target_edlp_spend_lower_bd[0])
    model.c7 = Constraint(expr = sum(edlp_trade_spent[0]) <= target_edlp_spend_upper_bd[0])
    
    model.c8 = Constraint(expr = sum(edlp_trade_spent[1]) >= target_edlp_spend_lower_bd[1])
    model.c9 = Constraint(expr = sum(edlp_trade_spent[1]) <= target_edlp_spend_upper_bd[1])
    
#     model.c1 = Constraint(expr = sum(total_trade_spent[0]) == target_trade_spend[0])
#     model.c2 = Constraint(expr = sum(total_trade_spent[1]) == target_trade_spend[1])
#     model.c3 = Constraint(expr = sum(edlp_trade_spent[0]) == target_edlp_spend[0] )
#     model.c4 = Constraint(expr = sum(edlp_trade_spent[1]) == target_edlp_spend[1] )
#     model.c5 = Constraint(expr = sum(calc_flag_util(model.flag))==0 )
    return model

In [6]:
import re
def get_info_from_results(results, info_string):
    i = str(results).lower().find(info_string.lower()) + len(info_string)
    value = ''
    while str(results)[i] != '\n':
        value = value + str(results)[i]
        i += 1
    return float(value)
def get_obj_frm_res(results, info_string):
#     print(results)
    results = results.lower()
    info_string = info_string.lower()
    z = re.search(info_string, results).start()
    
    return(float(results[z+108:z+125]))

In [8]:
def call_solver(init, name='bonmin'):
    assert type(init) == list
    model = create_model(init)
    opt_1 = SolverFactory(name)
    
#     opt_1.options['max_iter'] = 10
    
    results_1 = opt_1.solve(model)
    t = get_info_from_results(results_1, 'Time: ')
    print('Time taken is: ', t)
    model.display('hola.txt')
    file =  open('hola.txt', 'r')
    data = file.read()
    o = get_obj_frm_res(data, 'Objectives')
    file.close()
    print('Objective is: ', o)
    print('-------------------------------------------------')
    return o,t,model

combinations=[[0,0,0,0,0,0],
             [0,0,0,0,0,1],
             [0,0,0,0,1,1],
             [0,0,0,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,1],
             [1,0,0,0,1,1],
             [1,0,0,1,1,1],
             [1,1,0,0,0,0],
             [1,1,0,0,0,1],
             [1,1,0,0,1,1],
             [1,1,0,1,1,1],
             [1,1,1,0,0,0],
             [1,1,1,0,0,1],
             [1,1,1,0,1,1],
             [1,1,1,1,1,1]]    

time_lst = []
obj_lst = []
time_lst_1 = []
for c in combinations:
    start_time = time.time()
    t_lst, o_lst,model = call_solver(c, 'bonmin')
    end_time = time.time()
    time_lst.append(t_lst)
    obj_lst.append(o_lst)
    time_lst_1.append(end_time - start_time)
    model.display()
#     break
# time_lst, obj_lst
# 5566396.813079901

Init value=  [0, 0, 0, 0, 0, 0]
Time taken is:  1.375826358795166
Objective is:  5656058.269706419
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.662662504106965 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.808102160290161 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.673818381864061 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :     5 : 35.80181382143284 :    50 : False : False : NonNegativeReals
          1 :     5 : 16.10572497181227 :    50 : False : False : NonNegativeReals
          2 :     5 : 19.04758683501626 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale

Time taken is:  1.6262922286987305
Objective is:  5656058.269706419
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.662662504106965 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.808102160290161 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.673818381864061 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :     5 : 35.80181382143284 :    50 : False : False : NonNegativeReals
          1 :     5 : 16.10572497181227 :    50 : False : False : NonNegativeReals
          2 :     5 : 19.04758683501626 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :

Time taken is:  0.9905045032501221
Objective is:  5656058.269706417
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.673818381864061 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.662662504106963 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.754871434855136 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 : 35.754995976581355 :    50 : False : False : NonNegativeReals
          1 :     5 : 19.095253983178047 :    50 : False : False : NonNegativeReals
          2 :     5 :  16.10572497181228 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    

Time taken is:  0.11871337890625
Objective is:  5641638.501991752
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.672238692992039 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.672238692992039 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.792975326159208 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 :  49.22527899484841 :    50 : False : False : NonNegativeReals
          1 :     5 :  49.22527899484841 :    50 : False : False : NonNegativeReals
          2 :     5 : 16.457035645467123 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0

Time taken is:  0.5521762371063232
Objective is:  5656058.269706419
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.662662504106963 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.750067308806521 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.673818381864061 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 :  26.27820932598918 :    50 : False : False : NonNegativeReals
          1 :     5 : 16.105724971812275 :    50 : False : False : NonNegativeReals
          2 :     5 : 28.743955131310692 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    

Time taken is:  0.9599459171295166
Objective is:  5656058.269706419
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.662662504106963 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.750067308806521 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.673818381864061 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 :  26.27820932598918 :    50 : False : False : NonNegativeReals
          1 :     5 : 16.105724971812275 :    50 : False : False : NonNegativeReals
          2 :     5 : 28.743955131310692 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    

In [12]:
pd.DataFrame(data=[time_lst,obj_lst, time_lst_1]).to_csv('out_1.csv')
!cp out_1.csv /mnt/d/1/

In [7]:
pd.DataFrame(data=[time_lst,obj_lst, time_lst_1])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,566396.813080,566396.813080,566396.813080,566396.813080,566870.584722,566870.584722,566870.584722,566870.584722,566870.584722,566870.584722,566870.584722,566870.584722,566396.813080,566396.813080,566396.813080,566396.813080
1,0.104964,0.097383,0.103422,0.099730,0.156538,0.141058,0.152522,0.140854,0.319028,0.325191,0.310962,0.319351,0.136575,0.125980,0.124108,0.122236
2,0.656318,0.122395,0.128887,0.124074,0.183012,0.166600,0.177215,0.168876,0.347958,0.353341,0.337862,0.345281,0.164037,0.151053,0.148844,0.147053


In [ ]:
0.128878

In [50]:
def cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    
    unit_sales = []
    for i in  range(weeks):
#         print(edlp_p1[i],coef[0],flag_p1[i])
        a = (np.log( edlp_p1[i])*coef[0])* flag_p1[i]
        b = (np.log(base_price_1[i])*coef[0]+ tpr_p1[i]*coef[1])*(1- flag_p1[i])
        
        c = (np.log( edlp_p2[i])*coef[2])* flag_p2[i]
        d = (np.log(base_price_2[i])*coef[2]+ tpr_p2[i]*coef[3])*(1- flag_p2[i])
        
#         print(a,' $ ',b,' $ ',c,' $ ',d,' $ ',coef[4])
        unit_sales.append(np.exp(a+b+c+d+coef[4]))                 
   
    return unit_sales

def calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j):
    price = []
    edlp = [edlp_p1,edlp_p2]
    tpr = [tpr_p1,tpr_p2]
    flag = [flag_p1,flag_p2]
    base_price=[base_price_1,base_price_2]
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i]/100)*(1-flag[j][i]))  +  (edlp[j][i]*flag[j][i]))
                   
    return price
                   
                   
def calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    dollar_sales=[]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    edlp_trade_spent = []
    flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


In [51]:
products=2
coef_p1 = [-6,0.01,1,-0.003,23]
coef_p2 = [1.2,-0.004,-5,0.02,20]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
flag_p1=[1,0,0]
flag_p2=[0,0,1]

# cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef[0]),cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef[1])
# calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,0),calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,1)
# calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)
# calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)
# calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)

[[63366.278926377876, 0.0, 0.0], [0.0, 0.0, 35525.139769282396]]